In [1]:
import pymatgen
import numpy as np
from mp_api.client import MPRester
import re
import pandas as pd
import torch
import qmpy_rester as qr

In [2]:
# Datasets
# Really only using Hamidieh but just in case.
supercon_path = 'datasets/superconraw/primary.tsv'
hamidieh_path_data = 'datasets/Hamidieh; A Data-Driven Statistical Model for Predicting the Critical Temperature of a Superconductor/train.csv'
hamidieh_path_names = 'datasets/Hamidieh; A Data-Driven Statistical Model for Predicting the Critical Temperature of a Superconductor/unique_m.csv'
supercon_raw = np.genfromtxt(supercon_path, delimiter='\t')
supercondata_path = 'datasets/supercondata/220808_MDR_OAndM.txt'

# Open the text file and read its contents with the appropriate encoding
with open(supercondata_path, 'r', encoding='latin-1') as file:
    # Read the lines of the file
    lines = file.readlines()

# Process the lines and split into columns
supercondata_raw = []
for line in lines:
    columns = line.split('\t')  # Split each line by tab
    supercondata_raw.append(columns)

supercondata_raw = np.array(supercondata_raw)

In [3]:
hamidieh_data = pd.read_csv(hamidieh_path_data)
hamidieh_names = pd.read_csv(hamidieh_path_names)
periodictable = pd.read_csv('PeriodicTableCleaned_data.csv')
binaryoxides = np.genfromtxt('binaryoxide_data.csv', delimiter=',', dtype=None, encoding=None)

supercon_formations = []
import csv
# Open the CSV file
with open('supercon_formationE.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        supercon_formations.append(row)

# Convert the data into a NumPy array
supercon_formations = np.array(supercon_formations)

C:\Users\songo\AppData\Local\Temp\ipykernel_39744\1742257389.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  supercon_formations = np.array(supercon_formations)


In [4]:
# pnoxides = ['Li2O','Na2O','Al2O3','K2O','CaO','V2O5','Cr2O3','Mn2O3','BeO','MgO','Sc2O3','TiO2','Fe2O3','Co3O4','NiO','CuO','ZnO','Ga2O3','Rb2O','SrO','Y2O3',
#             'ZrO2', 'Nb2O5','MoO3','RuO2','Rh2O3','PdO','AgO','CdO','In2O3','SnO2','Cs2O','BaO','HfO2','Ta2O5','WO3','Re2O7','OsO2','IrO2','PtO','Au2O3','HgO'
#             ,'Tl2O3','PbO2','Bi2O3','La2O3','CeO2','Pr6O11','Nd2O3','Sm2O3','Eu2O3','Gd2O3','Tb4O6','Dy2O3','Ho2O3','Er2O3','Tm2O3','Yb2O3','Lu2O3','ThO2'
#             ,'UO2']

# #chatgpt common oxides for each element
# likliestoxides = ['Li2O', 'BeO', 'Na2O', 'MgO', 'Al2O3', 'SiO2', 'P4O10', 'SO3', 'Cl2O7', 'K2O', 'CaO', 'Sc2O3', 'TiO2', 'V2O5', 'Cr2O3', 'MnO2', 'Fe2O3', 'CoO', 'NiO', 'CuO', 'ZnO', 'Ga2O3', 'GeO2', 'As2O5', 'SeO2', 'Br2O5', 'Rb2O', 'SrO', 'Y2O3', 'ZrO2', 'Nb2O5', 'MoO3', 'TcO3', 'RuO2', 'Rh2O3', 'PdO', 'Ag2O', 'CdO', 'In2O3', 'SnO2', 'Sb2O5', 'TeO2', 'I2O5', 'Cs2O', 'BaO', 'La2O3', 'CeO2', 'Pr6O11', 'Nd2O3', 'PmO2', 'Sm2O3', 'Eu2O3', 'Gd2O3', 'Tb4O7', 'Dy2O3', 'Ho2O3', 'Er2O3', 'Tm2O3', 'Yb2O3', 'Lu2O3', 'HfO2', 'Ta2O5', 'W3O8', 'Re2O7', 'OsO4', 'IrO3', 'PtO2', 'Au2O3', 'HgO', 'Tl2O', 'PbO', 'Bi2O3', 'ThO2', 'Pa2O5', 'U3O8', 'NpO2', 'PuO2', 'Am2O3', 'Cm2O3', 'Bk2O3', 'Cf2O3', 'Es2O3', 'Fm2O3', 'Md2O3', 'No2O3', 'Lr2O3', 'RfO2', 'DbO2', 'SgO2', 'BhO2', 'HsO2', 'MtO2', 'DsO2', 'RgO2', 'CnO2', 'Nh2O3', 'Fl2O3', 'Mc2O3', 'Lv2O3', 'Ts2O3', 'Og2O3']
# likliestoxides_multiple = ['Li2O', 'BeO', 'Na2O', 'MgO', 'Al2O3', 'SiO2', 'P4O10', 'SO3', 'Cl2O7', 'K2O', 'CaO', 'Sc2O3', 'TiO2', 'V2O5', 'Cr2O3', 'MnO2', 'Fe2O3', 'CoO', 'NiO', 'CuO', 'ZnO', 'Ga2O3', 'GeO2', 'As2O5', 'SeO2', 'Br2O5', 'Rb2O', 'SrO', 'Y2O3', 'ZrO2', 'Nb2O5', 'MoO3', 'TcO3', 'RuO2', 'Rh2O3', 'PdO', 'Ag2O', 'CdO', 'In2O3', 'SnO2', 'Sb2O5', 'TeO2', 'I2O5', 'Cs2O', 'BaO', 'La2O3', 'CeO2', 'Pr6O11', 'Nd2O3', 'PmO2', 'Sm2O3', 'Eu2O3', 'Gd2O3', 'Tb4O7', 'Dy2O3', 'Ho2O3', 'Er2O3', 'Tm2O3', 'Yb2O3', 'Lu2O3', 'HfO2', 'Ta2O5', 'W3O8', 'Re2O7', 'OsO4', 'IrO3', 'PtO2', 'Au2O3', 'HgO', 'Tl2O', 'PbO', 'Bi2O3', 'ThO2', 'Pa2O5', 'U3O8', 'NpO2', 'PuO2', 'Am2O3', 'Cm2O3', 'Bk2O3', 'Cf2O3', 'Es2O3', 'Fm2O3', 'Md2O3', 'No2O3', 'Lr2O3', 'RfO2', 'DbO2', 'SgO2', 'BhO2', 'HsO2', 'MtO2', 'DsO2', 'RgO2', 'CnO2', 'Nh2O3', 'Fl2O3', 'Mc2O3', 'Lv2O3', 'Ts2O3', 'Og2O3', 'Li2O2', 'BeO2', 'Na2O2', 'MgO2', 'AlO', 'SiO', 'P2O3', 'SO2', 'Cl2O', 'K2O2', 'CaO2', 'ScO3', 'TiO', 'V2O3', 'CrO3', 'MnO', 'FeO', 'Co2O3', 'Ni2O3', 'Cu2O', 'ZnO2', 'GaO', 'GeO', 'As2O3', 'SeO3', 'Br2O3', 'Rb2O3', 'SrO2', 'YO', 'ZrO', 'NbO2', 'MoO2', 'TcO2', 'RuO', 'RhO3', 'Pd2O3', 'Ag2O3', 'CdO2', 'InO2', 'SnO', 'Sb2O3', 'TeO3', 'I2O3', 'Cs2O2', 'BaO2', 'LaO3', 'Ce2O3', 'Pr2O3', 'Nd2O5', 'Pm2O3', 'Sm2O3', 'Eu2O', 'GdO3', 'Tb2O3', 'Dy2O', 'Ho2O', 'Er2O', 'Tm2O', 'Yb2O', 'Lu2O', 'HfO', 'Ta2O3', 'WO3', 'Re2O5', 'Os2O3', 'IrO', 'PtO', 'AuO', 'Hg2O', 'Tl2O3', 'PbO2', 'BiO3', 'ThO', 'PaO2', 'UO2', 'Np2O3', 'Pu2O3', 'Am2O', 'Cm2O', 'Bk2O', 'Cf2O', 'Es2O', 'Fm2O', 'Md2O', 'No2O', 'Lr2O', 'RfO', 'DbO', 'SgO', 'BhO', 'HsO', 'MtO', 'DsO', 'RgO', 'CnO', 'Nh2O', 'Fl2O', 'Mc2O', 'Lv2O', 'Ts2O', 'Og2O']

# len(likliestoxides)


In [89]:
material_dataset = []

mainingredient = ['Superconductor'] ###
calculation1 = ['Formation Energy of supercon']
calculation2 = ['Weighted formation energy of likely oxides']

maincomp = ['atomicradius1','atomicradius2','atomicradius3','atomicradius4','atomicradius5','atomicradius6','atomicradius7','atomicradius8','atomicradius9','atomicradius10',
            'fusionheat1','fusionheat2','fusionheat3','fusionheat4','fusionheat5','fusionheat6','fusionheat7','fusionheat8','fusionheat9','fusionheat10',
            'thermalcond1','thermalcond2','thermalcond3','thermalcond4','thermalcond5','thermalcond6','thermalcond7','thermalcond8','thermalcond9','thermalcond10',
            'numvalence1','numvalence2','numvalence3','numvalence4','numvalence5','numvalence6','numvalence7','numvalence8','numvalence9','numvalence10',
            'Tc'] ###

comp1 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp2 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp3 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp4 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp5 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp6 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp7 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp8 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
comp9 = ['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
ingredient_coefficients = ['comp1','comp2','comp3','comp4','comp5','comp6','comp7','comp8','comp9']
all_comp = np.concatenate((ingredient_coefficients,comp1,comp2,comp3,comp4,comp5,comp6,comp7,comp8,comp9))




In [90]:
#### Make this very big data table. God this is a pain.
### Atomic radius is 21:31      Fusion heat is 
maincomp_atomicradius = np.array(hamidieh_data.iloc[:, 21:31])
maincomp_fusionheat = np.array(hamidieh_data.iloc[:, 51:61])
maincomp_thermalcond = np.array(hamidieh_data.iloc[:, 61:71])
maincomp_numvalence = np.array(hamidieh_data.iloc[:, 71:81])
maincomp_Tc = np.array(hamidieh_data.iloc[:,81])

#### RESERVED FOR CLEAN COMPONENT DATA ONLY
mainingredient_data = np.array(hamidieh_names.iloc[:,-1])
maincomp_data = np.column_stack((maincomp_atomicradius,maincomp_fusionheat,maincomp_thermalcond,maincomp_numvalence,maincomp_Tc))
#### EASY DATA IS DONE HERE
calculation1 = [element[-2] for element in supercon_formations[1:len(supercon_formations)]]




In [91]:
# Boot up materials project
# api_key = 'Lv3H4D1twE2kDiXZLWdaBTIRM82iRdCi'
# mpr = MPRester(api_key)

#### OBTAIN calculation1
def getbinaryoxides(element):
    oxides_data = np.asarray([999999,999999,999999])
    checker = np.asarray([999999,999999,999999])
    chars = len(element)
    for oxides in binaryoxides:
        if element == oxides[1][0:chars] and oxides[1][chars:chars+1].isdigit():
            oxidename = oxides[1]
            oxideformE = oxides[2]
            oxideEperatom = oxides[3]
            oxides_data = np.vstack((oxides_data, [oxidename, oxideformE, oxideEperatom]))
    
    if np.shape(oxides_data) == np.shape(checker):
        return [0]
    else:
        rank_indices = np.argsort(oxides_data[:,1])
        rankedoxides = np.flip(oxides_data[rank_indices], axis=0)
        return rankedoxides

### End output here is ranked_neg_oxides with the most likely oxide first.
def weight_exp(arr):
    weighted_arr = []
    # Weight using a logistic function
    supremum = np.max(arr)
    exp = len(arr) / 2
    x0 = np.average(arr) - np.std(arr)
    for energies in arr:
        weight = supremum/(1+np.exp(exp*(energies-x0))) * -1
        weighted_arr.append(weight)    
        
    return weighted_arr

#### STORE calculation 1


In [92]:
### Create list of element oxides and their formation energies + weights

elements = np.asarray(periodictable['symbol'])
elements_data = np.asarray(periodictable.iloc[:, 1:])
element_oxides = []

for sample in elements:
    test = getbinaryoxides(sample) 
    if np.shape(test) == np.shape([0]):
        element_oxides.append([0])
        continue
    list = test
    neg_oxides_name = []
    neg_oxides_formE = []
    neg_oxides_Eperatom = []

    ### Get rid of positive formation energy oxides.
    for oxides in list:
        if float(oxides[1]) < 0:
            neg_oxides_name.append(oxides[0])
            neg_oxides_formE.append(oxides[1])
            neg_oxides_Eperatom.append(oxides[2])

    neg_oxides = np.column_stack((neg_oxides_name, neg_oxides_formE, neg_oxides_Eperatom))
            
    uniques = np.unique(neg_oxides[:,0])
    list_uniques = []
    for oxides in uniques:
        formationE = neg_oxides[neg_oxides[:, 0] == oxides]
        likelyoxide = [formationE[0][0], formationE[0][1], formationE[0][2]]
        list_uniques.append(likelyoxide)
        # make into numpy array
    list_uniques = np.asarray(list_uniques)
    if len(list_uniques) == 0:
        element_oxides.append([0])
        continue
        
    ### list_uniques is the thing we want
    rank_list = np.argsort(list_uniques[:,1])
    ranked_neg_oxides = np.flip(list_uniques[rank_list], axis=0)
    ranked_neg_oxides_formE = np.asarray(ranked_neg_oxides[:,1], dtype = float)
    
    raw_weights_oxides = weight_exp(ranked_neg_oxides_formE)
    normalized_weights = raw_weights_oxides / np.sum(raw_weights_oxides)
    
    ####### THIS PART MAY NEED UPDATING IF WE DECIDE TO INCLUDE TERNARY OXIDES
    coeff_first = []
    coeff_second = []
    for oxides in ranked_neg_oxides:
        formula = oxides[0]
        coeff_first.append(re.search(r'\d+', formula).group())
        if formula[-1].isdigit():
            coeff_second.append(re.findall(r'\d+', formula)[1])
        else:
            coeff_second.append(1)
    
    coeff_first = np.asarray(coeff_first, dtype=float)
    coeff_second = np.asarray(coeff_second, dtype=float)
    netformE = np.asarray(ranked_neg_oxides[:,1], dtype=float) * (coeff_first + coeff_second)
    
    ### likely_oxides => Formatted as [Oxide name, Formation Energy / atom, Energy per atom, net formE, Weights, numfirstatoms]
    likely_oxides = np.column_stack((ranked_neg_oxides, netformE, normalized_weights, coeff_first))
    element_oxides.append(likely_oxides)


In [93]:
### Using this function takes the approach - 
## Assume that X% of element is allocated to this alloy, X% to the other alloy, X% to another...
# Then net formation energy is actual # of alloy made * net formation energy of alloy 

def componentformE_mass(element, size):
    indexarr = np.where(elements == element)
    index = indexarr[0][0]
    oxides = element_oxides[index]
    if oxides[0] == 0:
        return 0
    numelements_inoxide = np.asarray(oxides[:,5], dtype = float)
    weights = np.asarray(oxides[:,4], dtype = float)
    netformE = np.asarray(oxides[:,3], dtype = float)
    
    allocated_element = size * weights
    weighted_oxides = allocated_element / numelements_inoxide
    formE_oxides = weighted_oxides * netformE
    
    partialformE = np.sum(formE_oxides)
    return partialformE

### The other option is that X% of the TOTAL oxides made is 1, X% of the total is 2, ...
## Procedure is:
# First, based on coeff_first of the alloy, determine how much total hydrogen is required from percentages.
# So if there's 100 alloys total, X% * coeff_first[0] + X% * coeff_first[1] + ... = net # elements
#

def componentformE_total(element,size):
    indexarr = np.where(elements == element)
    index = indexarr[0][0]
    oxides = element_oxides[index]
    if oxides[0] == 0:
        return 0
    numelements_inoxide = np.asarray(oxides[:,5], dtype = float)
    weights = np.asarray(oxides[:,4], dtype = float)
    netformE = np.asarray(oxides[:,3], dtype = float)
    
    netelement = np.sum(weights*numelements_inoxide)
    multiplier = float(size) / float(netelement)
    formE_oxides = multiplier*netformE*weights
    
    partialformE = np.sum(formE_oxides)
    return partialformE


In [94]:
### Remember the formation energies of the oxides (weighted) is calculation2
calculation2 = []

import chemparse

for ingredient in mainingredient_data:
    parsedformula = chemparse.parse_formula(ingredient)
    parsedelements = [*parsedformula.keys()]
    parsedcomp = [*parsedformula.values()]
    parsed_main = np.column_stack((parsedelements,parsedcomp))
    calc_formE = []
    for components in parsed_main:
        element = components[0]
        size = components[1]
        formE = componentformE_total(element, size)
        calc_formE.append(formE)
    weightedformE = np.sum(np.asarray(calc_formE, dtype=float))
    calculation2.append(weightedformE)
    
### after running, calculation2 is now set. 

C:\Users\songo\AppData\Local\Temp\ipykernel_39744\44576258.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if oxides[0] == 0:


In [95]:
elements = np.asarray(periodictable['symbol'])
elements_data = np.asarray(periodictable.iloc[:, 1:])
index = np.where(elements == 'Cu')[0][0]
index
dat = [5]
test = []
test = np.concatenate((test, dat))
test = np.concatenate((test, dat))

test

array([5., 5.])

In [96]:
### Now populate the component data. We leave those bois blank if there aren't enough components.T
all_comp = []
for ingredient in mainingredient_data:
    parsedformula = chemparse.parse_formula(ingredient)
    parsedelements = [*parsedformula.keys()]
    parsedcomp = [*parsedformula.values()]
    parsed_main = np.column_stack((parsedelements,parsedcomp))
    
    complen = len(comp1)*9
    compfill = len(parsedelements)*13
    coefflen = len(ingredient_coefficients)
    coeff_fill = len(parsedelements)
    comp_unfilled = complen - compfill
    coeff_unfilled = coefflen - coeff_fill
    
    unfilled_comp = np.zeros(comp_unfilled)
    unfilled_coeff = np.zeros(coeff_unfilled)
    filled_comp = []
    filled_coeff = []
    for parsed in parsed_main:
        element = parsed[0]
        weight = [parsed[1]]
        index = np.where(elements == element)[0][0]
        elementdata = elements_data[index]

        filled_coeff = np.concatenate((filled_coeff, weight))
        filled_comp = np.concatenate((filled_comp, elementdata))
        
    net_comp = np.concatenate((filled_comp, unfilled_comp))
    net_coeff = np.concatenate((filled_coeff, unfilled_coeff))
    
    row = np.concatenate((net_coeff, net_comp))
    
    all_comp.append(row)
    
    

In [102]:
## test - all first shape elements should be the same
print(np.shape(mainingredient_data))
print(np.shape(calculation1))
print(np.shape(calculation2))
print(np.shape(maincomp_data))
print(np.shape(all_comp))

(21263,)
(21263,)
(21263,)
(21263, 41)
(21263, 126)


In [104]:
material_dataset_data = np.column_stack((mainingredient_data, calculation1, calculation2, maincomp_data, all_comp))

In [106]:
np.savetxt('material_dset_data.csv', material_dataset_data, delimiter=',',fmt='%s')
